In [45]:
import boto3 # Python integration with AWS.
import pandas as pd # Read csv.
from io import StringIO, BytesIO # StringIO to read csv on AWS. BytesIO to save as Parquet file to S3.
from datetime import datetime, timedelta # Date functions.

In [46]:
# Argument Date for start of report
arg_date = '2022-11-22'
# Moving all arguments and parameters to the beggining of the code.
scr_format = '%Y-%m-%d'
scr_bucket = 'xetra-1234'
trg_bucket = 'manoetl123'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
# Write to S3.
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [47]:
# Convert to data type and use previous day
arg_date_dt = datetime.strptime(arg_date, scr_format).date() - timedelta(days=1)

In [48]:
# Variables for data on AWS S3.
s3 = boto3.resource('s3')
bucket = s3.Bucket(scr_bucket)
# List of filtered CSVs.
# Filter using arg_date
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], scr_format).date() >= arg_date_dt]

In [49]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    # Read csvs with StringIO (needed for AWS) and Pandas
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

In [50]:
# DF with filtered columns.
df_all = df_all.loc[:, columns]

In [51]:
# Filter out empty lines.
df_all.dropna(inplace=True)

# Get opening price per ISIN and day

In [52]:
# Get opening price per ISIN and day
# Create new column with opening price.
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

Get closing price per ISIN and day

In [53]:
# Get closing price per ISIN and day
# Create new column with closing price.
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [54]:
# Aggregations
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

## Percent Change Prev Closing

In [55]:
# Percent Change Prev Closing
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [56]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [57]:
# Remove column and update df_all
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [58]:
# Round everything.
df_all = df_all.round(decimals=2)

In [59]:
# Filter by date, don't show previous date that was used for calculation.
df_all = df_all[df_all.Date >= arg_date]

## Write to S3.

In [60]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='manoetl123', key='xetra_daily_report_20221122_233442.parquet')

In [61]:
# Read the uploaded file.
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20221122_222801.parquet
xetra_daily_report_20221122_233442.parquet


In [64]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20221122_233442.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [65]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-11-22,36.40,36.85,36.40,36.85,1475,-1.60
1,AT00000FACC2,2022-11-22,7.89,8.47,7.89,8.47,1956,1.80
2,AT0000606306,2022-11-22,21.36,21.78,21.20,22.62,61546,-8.26
3,AT0000609607,2022-11-22,12.20,12.30,12.14,12.32,930,-1.28
4,AT0000644505,2022-11-22,98.40,100.80,98.40,102.20,2413,-1.56
...,...,...,...,...,...,...,...,...
3248,XS2284324667,2022-11-22,38.72,38.33,38.27,39.07,4245,0.64
3249,XS2314659447,2022-11-22,8.74,8.76,8.71,8.81,133,0.22
3250,XS2314660700,2022-11-22,22.10,22.04,22.04,22.10,239,1.60
3251,XS2376095068,2022-11-22,32.47,33.04,32.47,33.22,758,-3.36
